# RNN-LSTMs in Keras Demo with IMDB data

## This is my own RNN-LSTM to predict the last word in a sentence

In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.optimizers import RMSprop
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
%matplotlib inline

Using Theano backend.
Using gpu device 0: GeForce GT 750M (CNMeM is disabled, cuDNN 5004)

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
max_features = 500
emb_dim = 10
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)

# Taking the last element of X_train[i] as our outcome
Y_train = np.asarray([i[-1] for i in X_train]) 
Y_test = np.asarray([ i[-1] for i in X_test])

# Removing the last element of X_train[i]
X_train = np.asarray([i[:-1] for i in X_train]) 
X_test = np.asarray([i[:-1] for i in X_test] )


windx = imdb.get_word_index()
rwindx = dict((k,v) for v,k in windx.items())

# Proof it the index swap worked
print( windx['fawn'])
print( rwindx[34701])

# Looking at one set of data
print( X_train[0])

# Let's see the words
print( [rwindx[x] for x in X_train[0]])

34701
fawn
[1, 14, 22, 16, 43, 2, 2, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 2, 112, 50, 2, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 2, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 2, 38, 76, 15, 13, 2, 4, 22, 17, 2, 17, 12, 16, 2, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 2, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 2, 36, 71, 43, 2, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178]
['the', 'as', 'you', 'with', 'out', 'and', 'and', 'and', 'and', 'their', 'becomes', 'and', 'had', 'and', 'of', 'lot', 'from', 'anyone', 'to', 'h

In [3]:
# RNN for the regular embedings
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
# The pad sequences stuff seems suspicious
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

25000 train sequences
25000 test sequences
Pad sequences (samples x time)
X_train shape: (25000, 80)
X_test shape: (25000, 80)


In [5]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, emb_dim, dropout=0.2))
model.add(LSTM(150))
model.add(Dense(max_features))
model.add(Activation('softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

optimizer = RMSprop(lr=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer)

print('Train...')
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=1,
          validation_data=(X_test, Y_test), verbose=0)

# Running the predictions
out = model.evaluate(X_test, Y_test, batch_size=batch_size)
ypred_tst = model.predict(X_test)

print('Cross Entropy Loss of %0.3f' % out)

yprdtst = np.argmax(ypred_tst, axis=1)
x = confusion_matrix(yprdtst, Y_test).astype('float') 
# Normalizing it
x_n = x/ x.sum(axis=0)

print ('Classification Accuracy is %0.3f' % (np.diag(x).sum()/ x.sum()))

Build model...
Train...
25000/25000 [==============================] - 53s    
Cross Entropy Loss of 3.005
Classification Accuracy is 0.516


In [ ]:
# Look at the confusion matrix image
plt.figure(figsize=(15,10))
plt.imshow(x_n, cmap='OrRd')
plt.show()

In [ ]:
len(model.get_weights())

In [ ]:
np.unique(X_train), len(np.unique(X_train)), len(range(np.min(X_train), np.max(X_train)+1)) # Have to do +1

In [ ]:
model.get_weights()[0].shape

In [ ]:
[ [j.shape for j in l.get_weights()] for l in model.layers]

In [ ]:
# Embedding layers
embs = model.layers[0].get_weights()[0]
embs.shape

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(embs.dot(embs.T), cmap='OrRd')
plt.show()

In [ ]:
uniqs = np.unique(X_train)
min_x, max_x = np.min(X_train), np.max(X_train)
print(uniqs)
print("%i unique values"% len(uniqs) )
print('Ranging from %i to %i' % (min_x, max_x) )
if min_x==0:
    max_x+=1
print("Which means we have an index matrix with %i words" % (max_x-min_x) )

In [ ]:
# The indexing below is correct
# -- note that we don't have a 3 in our np.unique(X_train) so we only have 40 unique values b/c we start at 0
# -- which is inserted by the padding function, and note that we don't have 0 in our dictionary list
# --
sims = embs.dot( embs[windx['film']].reshape((10,1)))
for i, si in enumerate(sims):
    if i==0:
        continue
    print(i, rwindx[i], si)
        
print('Closest match is %i' % np.delete(sims, windx['film']).argmax())

In [ ]:
rwindx[3]

In [ ]:
# Here are the words in the model
print( [rwindx[x] for x in np.unique(X_train).tolist() if x!=0])

----

## This is the original RNN-LSTM-MLP for IMDB Classification {0,1}

Original Source: https://github.com/fchollet/keras/blob/master/examples/imdb_lstm.py

    from __future__ import print_function
    import numpy as np
    np.random.seed(1337)  # for reproducibility

    from keras.preprocessing import sequence
    from keras.utils import np_utils
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, Activation, Embedding
    from keras.layers import LSTM, SimpleRNN, GRU
    from keras.datasets import imdb
    from keras.optimizers import RMSprop
    from sklearn.metrics import roc_auc_score

    max_features = 200
    maxlen = 80  # cut texts after this number of words (among top max_features most common words)
    batch_size = 32

    # This is the original IMDB model, it's an LSTM feeding into an MLP

    print('Loading data...')

    (X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)

    print(len(X_train), 'train sequences')
    print(len(X_test), 'test sequences')

    print('Pad sequences (samples x time)')
    X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
    X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
    print('X_train shape:', X_train.shape)
    print('X_test shape:', X_test.shape)

    print('Build model...')
    model = Sequential()
    model.add(Embedding(max_features, 128, dropout=0.2))
    model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    # try using different optimizers and different optimizer configs
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    print('Train...')
    model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1,
              validation_data=(X_test, y_test), verbose=0)
    score, acc = model.evaluate(X_test, y_test,
                                batch_size=batch_size)
    print('Test score:', score)
    print('Test accuracy:', acc)
    ypred_tst = model.predict(X_test)
    # Pretty good AUC
    print('Test AUC:', roc_auc_score(y_test,ypred_tst))